## Download CMIP6 data from Pangeo

Variables of interest: `mrsos`

## Necessary Packages

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import cftime

from tqdm.notebook import tqdm
import re
import os

from operator import itemgetter # For list subsetting (very silly name)

import warnings

import intake
import intake_esm
import gcsfs

import matplotlib.pyplot as plt

In [2]:
# Get config file 
import download_config as cfg

In [3]:
cfg.lpaths['raw_data_dir']

'/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/'

In [4]:
# Set whether to regrid 360-day calendars to 365-day calendars
# (probably don't do this while saving files. Only do this in 
# processing code that doesn't affect the original file)
regrid_360 = False

## Get list of cloud datasets

In [5]:
url = "https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog.json" # Only stores that pass current tests
col = intake.open_esm_datastore(url)

varlist = ['mrsos']

cats = {var:col.search(variable_id = [var], table_id = 'Lmon') for var in varlist}

### Turn into dataframe

In [6]:
cats['mrsos'].df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,sub_experiment_id,variant_label,version,zstore
0,CMIP,NASA-GISS,GISS-E2-1-G,piControl,r101i1p1f1,Lmon,mrsos,gn,none,r101i1p1f1,v20190815,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/pi...
1,C4MIP,NASA-GISS,GISS-E2-1-G,1pctCO2-bgc,r101i1p1f1,Lmon,mrsos,gn,none,r101i1p1f1,v20190815,gs://cmip6/CMIP6/C4MIP/NASA-GISS/GISS-E2-1-G/1...
2,CMIP,NASA-GISS,GISS-E2-1-G,piControl,r102i1p1f1,Lmon,mrsos,gn,none,r102i1p1f1,v20190815,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/pi...
3,ScenarioMIP,CCCma,CanESM5,ssp585,r6i1p2f1,Lmon,mrsos,gn,none,r6i1p2f1,v20190429,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...
4,ScenarioMIP,CCCma,CanESM5,ssp585,r25i1p1f1,Lmon,mrsos,gn,none,r25i1p1f1,v20190429,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...
...,...,...,...,...,...,...,...,...,...,...,...,...
884,ScenarioMIP,MOHC,UKESM1-0-LL,ssp126,r3i1p1f2,Lmon,mrsos,gn,none,r3i1p1f2,v20190507,gs://cmip6/CMIP6/ScenarioMIP/MOHC/UKESM1-0-LL/...
885,ScenarioMIP,IPSL,IPSL-CM6A-LR,ssp245,r2i1p1f1,Lmon,mrsos,gr,none,r2i1p1f1,v20190516,gs://cmip6/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR...
886,DAMIP,BCC,BCC-CSM2-MR,hist-nat,r2i1p1f1,Lmon,mrsos,gn,none,r2i1p1f1,v20190505,gs://cmip6/CMIP6/DAMIP/BCC/BCC-CSM2-MR/hist-na...
887,CMIP,MOHC,UKESM1-0-LL,historical,r8i1p1f2,Lmon,mrsos,gn,none,r8i1p1f2,v20190502,gs://cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/histori...


In [7]:
# Set CMIP6 data parameters: each of these dict keys 
# is a column in the dataframe that gives the link 
# of each of the relevant files. Files defined by
# each dict are processed separately. 
data_params_all =[{'experiment_id':'historical','table_id':'Lmon','variable_id':'mrsos','member_id':'r1i1p1f1'},
               #{'experiment_id':'ssp370','table_id':'Lmon','variable_id':'mrsos','member_id':'r1i1p1f1'},
               {'experiment_id':'ssp585','table_id':'Lmon','variable_id':'mrsos','member_id':'r1i1p1f1'}] 


## Subsetting

Set parameters for spatiotemporal subsetting

In [8]:
# Set parameters for spatiotemporal subsetting
subset_params_all = [{'lat':[-60,15],
                      'lon':[-85,-33],
                      'time':{'historical':['1979-01-01','2014-12-31'], # specify the experiment id for each time range
                              #'ssp370':['2015-01-01','2100-12-31'], 
                              'ssp585':['2015-01-01','2100-12-31'],
                             }, 
                 'fn_suffix':'_SouthAmerica', # added to end of filename when saving
                 'lon_range':180, # 180 or 360 - do you want your output file to count lon -180:180 or 0:360?
                 'lon_origin':-180}] # set origin (first lon value) of pre-processed grid. 

## `fix_lons` auxillary funciton

In [9]:
def fix_lons(ds,subset_params):
    """
    This function fixes a few issues that show up when dealing with 
    longitude values. 
    
    Input: an xarray dataset, with a longitude dimension called "lon"
    
    Changes: 
    - The dataset is re-indexed to -180:180 or 0:360 longitude format, 
      depending on the subset_params['lon_range'] parameter
    - the origin (the first longitude value) is changed to the closest 
      lon value to subset_params['lon_origin'], if using a 0:360 range. 
      In other words, the range becomes [lon_origin:360 0:lon_origin]. 
      This is to make sure the subsetting occurs in the 'right' direction, 
      with the longitude indices increasing consecutively (this is to ensure
      that subsetting to, say, [45, 275] doesn't subset to [275, 45] or vice-
      versa). Set lon_origin to a longitude value lower than your first subset 
      value.
    """

    ## save the attrs for l8rs
    tmp_attrs = ds.lon.attrs
    
    if subset_params['lon_range']==180:
        # Switch to -180:180 longitude if necessary
        if any (ds.lon>180):
            ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))
        # Change origin to half the world over, to allow for the 
        # longitude indexing to cross the prime meridian, but only
        # if the first lon isn't around -180 (using 5deg as an approx
        # biggest grid spacing). This is intended to move [0:180 -180:0]
        # to [-180:0:180].
        if ds.lon[0] > -175:
            ds = ds.roll(lon=(ds.sizes['lon'] // 2),roll_coords=True)
    elif subset_params['lon_range']==360:
        # Switch to 0:360 longitude if necessary
        ds = ds.assign_coords(lon = ds.lon % 360)
        # Change origin to the lon_origin
        ds = ds.roll(lon=-((ds.lon // subset_params['lon_origin'])==1).values.nonzero()[0][0],roll_coords=True)


    ds.lon.attrs = tmp_attrs
    return ds


# Process

In [10]:
# Define the catalog URL
url = "https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog.json"

# Open the ESM datastore
col = intake.open_esm_datastore(url)

# Define search criteria
varlist = ['mrsos']
table_id = 'Lmon'
experiment_ids = ['historical', 'ssp585']

# Search without specifying member_id 
df_filtered = col.search(
    variable_id=varlist,
    table_id=table_id,
    experiment_id=experiment_ids
).df

# find member_ids that appear in all experiments for each model (source_id)
valid_members = (
    df_filtered.groupby(['source_id', 'member_id'])['experiment_id']
    .apply(lambda exp_ids: set(experiment_ids).issubset(set(exp_ids)))  # Check if all experiments exist
)

# get valid (source_id, member_id) pairs
valid_pairs = valid_members[valid_members].index.tolist()

# keep only valid (source_id, member_id) combos
cmip6_sub = df_filtered[
    df_filtered.set_index(['source_id', 'member_id']).index.isin(valid_pairs)
]


In [11]:
#cmip6_sub = cmip6_sub[(cmip6_sub.source_id == 'UKESM1-0-LL') & (cmip6_sub.member_id == 'r1i1p1f2')]

In [12]:
#cmip6_sub 

In [13]:
# Set CMIP6 data parameters: each of these dict keys 
# is a column in the dataframe that gives the link 
# of each of the relevant files. Files defined by
# each dict are processed separately. 
data_params_all =[{'experiment_id':'historical','table_id':'Lmon','variable_id':'mrsos'},
               #{'experiment_id':'ssp370','table_id':'Lmon','variable_id':'mrsos'},
               {'experiment_id':'ssp585','table_id':'Lmon','variable_id':'mrsos'}] 


In [14]:
data_params_all

[{'experiment_id': 'historical', 'table_id': 'Lmon', 'variable_id': 'mrsos'},
 {'experiment_id': 'ssp585', 'table_id': 'Lmon', 'variable_id': 'mrsos'}]

In [15]:
#------ Process by variable and dataset in the subset ------
#overwrite=True
overwrite = False

for data_params in data_params_all:
    
    for url in tqdm(cmip6_sub.zstore.values):
        # Set output filenames
        output_fns = [None]*len(subset_params_all)
        path_exists = [None]*len(subset_params_all)

        #data_params['member_id'] = url.split('/')[8]
        for subset_params in subset_params_all:
            output_fns[subset_params_all.index(subset_params)] = (cfg.lpaths['raw_data_dir']+url.split('/')[6]+'/'+
                                                                 data_params['variable_id']+'_'+
                                                                 data_params['table_id']+'_'+url.split('/')[6]+'_'+
                                                                 data_params['experiment_id']+'_'+url.split('/')[8]+'_'+
                                                                 '-'.join([re.sub('-','',t) for t in subset_params['time'][data_params['experiment_id']]])+
                                                                 subset_params['fn_suffix']+'.nc')
            
            if 'other' in data_params.keys(): 
                if 'plev_subset' in data_params['other'].keys():
                    output_fns[subset_params_all.index(subset_params)] = re.sub(data_params['variable_id'],
                                                                            data_params['other']['plev_subset']['outputfn'],
                                                                           output_fns[subset_params_all.index(subset_params)])
            
            
            
            path_exists[subset_params_all.index(subset_params)] = os.path.exists(output_fns[subset_params_all.index(subset_params)])
        
        if (not overwrite) & all(path_exists):
            warnings.warn('All files already created for '+data_params['variable_id']+' '+
                                                                 data_params['table_id']+' '+url.split('/')[6]+' '+
                                                                 data_params['experiment_id']+' '+url.split('/')[8]+', skipped.')
            continue
        elif any(path_exists):
            if overwrite:
                for subset_params in subset_params_all:
                    if path_exists[subset_params_all.index(subset_params)]:
                        os.remove(output_fns[subset_params_all.index(subset_params)])
                        warnings.warn('All files already exist for '+data_params['variable_id']+' '+
                                                                             data_params['table_id']+' '+url.split('/')[6]+' '+
                                                                             data_params['experiment_id']+' '+url.split('/')[8]+
                                      ', because OVERWRITE=TRUE theses files have been deleted.')
        
        
        # Open dataset
        #ds = xr.open_zarr(fs.get_mapper(url),consolidated=True)
        ds = xr.open_zarr(url, consolidated = True)
        # ds.mrsos.mean(dim = 'time').plot()
        # plt.show()

        # Rename to lat / lon (let's hope there's no 
        # Latitude / latitude_1 / etc. in this dataset)
        try:
            ds = ds.rename({'longitude':'lon','latitude':'lat'})
        except: 
            pass
        
        # same with 'nav_lat' and 'nav_lon' ???
        try:
            ds = ds.rename({'nav_lon':'lon','nav_lat':'lat'})
        except: 
            pass

        # If precip, kg/m^2/s, switch to mm/day
        #if data_params['variable_id']=='pr':
        #    ds[data_params['variable_id']] = ds[data_params['variable_id']]*60*60*24

        # Fix coordinate doubling (this was an issue in NorCPM1, 
        # where thankfully the values of the variables were nans,
        # though I still don't know how this happened - some lat
        # values were doubled within floating point errors)
        if 'lat' in ds[data_params['variable_id']].dims:
            if len(np.unique(np.round(ds.lat.values,10))) != ds.dims['lat']:
                ds = ds.isel(lat=(~np.isnan(ds.isel(lon=1,time=1)[data_params['variable_id']].values)).nonzero()[0],drop=True)
                warnings.warn('Model '+ds.source_id+' has duplicate lat values; attempting to compensate by dropping lat values that are nan in the main variable in the first timestep')
            if len(np.unique(np.round(ds.lon.values,10))) != ds.dims['lon']:
                ds = ds.isel(lon=(~np.isnan(ds.isel(lat=1,time=1)[data_params['variable_id']].values)).nonzero()[0],drop=True)
                warnings.warn('Model '+ds.source_id+' has duplicate lon values; attempting to compensate by dropping lon values that are nan in the main variable in the first timestep')

        # Sort by time, if not sorted (this happened with
        # a model; keeping a warning, cuz this seems weird)
        if (ds.time.values != np.sort(ds.time)).any():
            warnings.warn('Model '+ds.source_id+' has an unsorted time dimension.')
            ds = ds.sortby('time')
            
        # If 360-day calendar, regrid to 365-day calendar
        if regrid_360:
            if ds.dims['dayofyear'] == 360:
                # Have to put in the compute() because these 
                # are by default dask arrays, chunked along
                # the time dimension, and can't interpolate
                # across dask chunks... 
                ds = ds.compute().interp(dayofyear=(np.arange(1,366)/365)*360)
                # And reset it to 1:365 indexing on day of year
                ds['dayofyear'] = np.arange(1,366)
                # Throw in a warning, too, why not
                warnings.warn('Model '+ds.source_id+' has a 360-day calendar; daily values were interpolated to a 365-day calendar')

        # Now, save by the subsets desired in subset_params_all above
        for subset_params in subset_params_all:
            # Make sure this file hasn't already been processed
            if (not overwrite) & path_exists[subset_params_all.index(subset_params)]:
                warnings.warn(output_fns[subset_params_all.index(subset_params)]+' already exists; skipped.')
                continue
            
            # Make sure the target directory exists
            if not os.path.exists(cfg.lpaths['raw_data_dir']+url.split('/')[6]+'/'):
                os.mkdir(cfg.lpaths['raw_data_dir']+url.split('/')[6]+'/')
                warnings.warn('Directory '+cfg.lpaths['raw_data_dir']+url.split('/')[6]+'/'+' created!')
         
            # Fix longitude (by setting it to either [-180:180] 
            # or [0:360] as determined by subset_params, and 
            # to roll them so the correct range is consecutive 
            # in lon (so if you're looking at the Equatorial 
            # Pacific, make it 0:360, with the first lon value
            # at 45E). 
            if 'lat' in ds[data_params['variable_id']].dims:
                ds_tmp = fix_lons(ds, subset_params)
                # Now, cutoff the values below the 'lon_origin', 
                # because slice doesn't work if the indices aren't
                # montonically increasing (and the above changes it
                # to [lon_origin:360 0:lon_origin]
                if np.abs(ds_tmp.lon[0]-subset_params['lon_origin'])>5:
                    ds_tmp = ds_tmp.isel(lon=np.arange(0,(ds_tmp.lon // (subset_params['lon_origin']) == 0).values.nonzero()[0][0]))
            else:
                ds_tmp = ds
                warnings.warn('fix_lons did not work because of the multi-dimensional index')

            # ds_tmp.mrsos.mean(dim = 'time').plot()
            # plt.show()
            
            # Subset by time as set in subset_params
            if (ds.time.max().dt.day==30) | (type(ds.time.values[0]) == cftime._cftime.Datetime360Day): 
                print('360-day')
                # (If it's a 360-day calendar, then subsetting to "12-31"
                # will throw an error; this switches that call to "12-30")
                # Also checking explicitly for 360day calendar; some monthly 
                # data is still shown as 360-day even when it's monthly, and will
                # fail on date ranges with date 31 in a month
                ds_tmp = (ds_tmp.sel(time=slice(subset_params['time'][data_params['experiment_id']][0],
                                        re.sub('-31','-30',subset_params['time'][data_params['experiment_id']][1]))))
            else:
                ds_tmp = (ds_tmp.sel(time=slice(*subset_params['time'][data_params['experiment_id']])))
            
           # Subset by space as set in subset_params
            if not 'lat' in ds[data_params['variable_id']].dims:
                ds_tmp = ds_tmp.where((ds_tmp.lat >= subset_params['lat'][0]) & (ds_tmp.lat <= subset_params['lat'][1]) &
                 (ds_tmp.lon >= subset_params['lon'][0]) & (ds_tmp.lon <= subset_params['lon'][1]),drop=True)
            else:
                ds_tmp = (ds_tmp.sel(lat=slice(*subset_params['lat']),
                                     lon=slice(*subset_params['lon'])))
            
            
            #ds_tmp.mrsos.mean(dim = 'time').plot()   
            
            # If subsetting by pressure level...
            if 'other' in data_params.keys():
                if 'plev_subset' in data_params['other'].keys:
                    # Have to use np.allclose for floating point errors
                    try:
                        ds_tmp = ds_tmp.isel(plev=np.where([np.allclose(p,data_params['other']['plev_subset']['plev']) for p in ds_tmp.plev])[0][0])
                        ds_tmp = ds_tmp.rename({data_params['variable_id']:data_params['other']['plev_subset']['outputfn']})
                    except KeyError:
                        print('The pressure levels: ')
                        print(ds_tmp.plev.values)
                        print(' do not contain '+str(data_params['other']['plev_subset']['plev'])+'; skipping.')
                        del ds_tmp
                        continue

            # Save as NetCDF file
            ds_tmp.to_netcdf(output_fns[subset_params_all.index(subset_params)])
            plt.show()

            # Status update
            print(output_fns[subset_params_all.index(subset_params)]+' processed!')
        
        del ds, ds_tmp, subset_params
                

  0%|          | 0/160 [00:00<?, ?it/s]

/var/folders/72/gq_qh34d21xgnsw2_5sf25rh0000gn/T/ipykernel_43876/3718736682.py:41: UserWarning: All files already exist for mrsos Lmon CanESM5 historical r6i1p2f1, because OVERWRITE=TRUE theses files have been deleted.
  warnings.warn('All files already exist for '+data_params['variable_id']+' '+


/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/CanESM5/mrsos_Lmon_CanESM5_historical_r6i1p2f1_19790101-20141231_SouthAmerica.nc processed!
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/CanESM5/mrsos_Lmon_CanESM5_historical_r25i1p1f1_19790101-20141231_SouthAmerica.nc processed!
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/CanESM5/mrsos_Lmon_CanESM5_historical_r24i1p2f1_19790101-20141231_SouthAmerica.nc processed!
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/CanESM5/mrsos_Lmon_CanESM5_historical_r24i1p1f1_19790101-20141231_SouthAmerica.nc processed!
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/CanESM5/mrsos_Lmon_CanESM5_historical_r25i1p2f1_19790101-20141231_SouthAmerica.nc processed!
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/CanESM5/mrsos_Lmon_CanESM5

/var/folders/72/gq_qh34d21xgnsw2_5sf25rh0000gn/T/ipykernel_43876/3718736682.py:111: UserWarning: Directory /Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/IPSL-CM6A-LR/ created!
  warnings.warn('Directory '+cfg.lpaths['raw_data_dir']+url.split('/')[6]+'/'+' created!')


/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/IPSL-CM6A-LR/mrsos_Lmon_IPSL-CM6A-LR_historical_r1i1p1f1_19790101-20141231_SouthAmerica.nc processed!


...

360-day
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/UKESM1-0-LL/mrsos_Lmon_UKESM1-0-LL_ssp585_r1i1p1f2_20150101-21001231_SouthAmerica.nc processed!


/var/folders/72/gq_qh34d21xgnsw2_5sf25rh0000gn/T/ipykernel_43876/3718736682.py:41: UserWarning: All files already exist for mrsos Lmon UKESM1-0-LL ssp585 r4i1p1f2, because OVERWRITE=TRUE theses files have been deleted.
  warnings.warn('All files already exist for '+data_params['variable_id']+' '+


360-day
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/UKESM1-0-LL/mrsos_Lmon_UKESM1-0-LL_ssp585_r4i1p1f2_20150101-21001231_SouthAmerica.nc processed!


/var/folders/72/gq_qh34d21xgnsw2_5sf25rh0000gn/T/ipykernel_43876/3718736682.py:41: UserWarning: All files already exist for mrsos Lmon UKESM1-0-LL ssp585 r8i1p1f2, because OVERWRITE=TRUE theses files have been deleted.
  warnings.warn('All files already exist for '+data_params['variable_id']+' '+


360-day
/Users/miriam/Documents/02_academia/02_PhD/research/projects/dirt_in_sesa/CMIP6_data/UKESM1-0-LL/mrsos_Lmon_UKESM1-0-LL_ssp585_r8i1p1f2_20150101-21001231_SouthAmerica.nc processed!
